## Final Project Submission

Please fill out:
* Student name: Amani Wanene
* Student pace:  full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


1. Import relevant resources

In [277]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import numpy as np


2. Import the datasets.

In [278]:
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz',encoding = 'latin1')
movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz',delimiter = '\t')
reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz',delimiter = '\t',encoding = 'latin1')
tmdb_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')


In [279]:


import zipfile

# Extract IMDb SQL .db file
with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:
    # Extract all contents of .zip file into current directory
    zipObj.extractall(path='./zippedData/')
    
# Create connection to IMDb DB


In [280]:
imdb_conn = sqlite3.connect('zippedData/im.db')


In [281]:
imdb_df = pd.read_sql("""SELECT * FROM movie_basics
 JOIN movie_ratings
USING(movie_id)
""",imdb_conn)

3.View the datasets

In [282]:
print(bom_df.shape)
bom_df.isna().sum()




(3387, 5)


title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [283]:
print(movie_info_df.shape)
movie_info_df.isna().sum()

(1560, 12)


id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [284]:
print(reviews_df.shape)
reviews_df.isna().sum()

(54432, 8)


id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

In [285]:
tmdb_df.isna().sum()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [286]:
movie_budgets_df.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [287]:
print(imdb_df.shape)
imdb_df.isna().sum()

(73856, 8)


movie_id              0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7620
genres              804
averagerating         0
numvotes              0
dtype: int64

4. Clean Data

A) bom_df

In [288]:
#drop rows with missing domestic_gross
bom_df = bom_df.dropna(subset=['domestic_gross'])

print(bom_df.loc[bom_df['domestic_gross'].isna()])

Empty DataFrame
Columns: [title, studio, domestic_gross, foreign_gross, year]
Index: []


In [289]:
#check percentage of foreign_gross missing values
bom_df['foreign_gross'].isna().sum()
missing_values_count = len(bom_df[bom_df['foreign_gross'].isna()])
total_rows = len(bom_df)

proportion_missing_values = missing_values_count / total_rows
print(proportion_missing_values)


0.40190532896695447


In [290]:
#Change null values to numeric in order to perform mathematical operations on the column.
bom_df['foreign_gross'] = pd.to_numeric(bom_df['foreign_gross'], errors='coerce')
bom_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3359 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3359 non-null   object 
 1   studio          3356 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2004 non-null   float64
 4   year            3359 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 157.5+ KB


In [291]:
#check the top 10 movies with missing foreign_domestic values
missing_foreign_gross_movies = bom_df[bom_df['foreign_gross'].isna()]
top_10_movies = missing_foreign_gross_movies.sort_values(by='domestic_gross', ascending=False).head(10)
print(top_10_movies)

                             title studio  domestic_gross  foreign_gross  year
1872  Star Wars: The Force Awakens     BV     936700000.0            NaN  2015
3079        Avengers: Infinity War     BV     678800000.0            NaN  2018
1873                Jurassic World   Uni.     652300000.0            NaN  2015
1874                     Furious 7   Uni.     353000000.0            NaN  2015
2760       The Fate of the Furious   Uni.     226000000.0            NaN  2017
3160                     Book Club   Par.      68600000.0            NaN  2018
1955                      War Room   TriS      67800000.0            NaN  2015
2864                All Eyez on Me   LG/S      44900000.0            NaN  2017
833                Red Dawn (2012)     FD      44800000.0            NaN  2012
2866                47 Meters Down  ENTMP      44300000.0            NaN  2017


In [292]:
#filter out movies with domestic gross < $10,000,000
filtered_movies = bom_df[bom_df['domestic_gross'] >= 10000000]
filtered_movies.isna().sum()

title              0
studio             0
domestic_gross     0
foreign_gross     58
year               0
dtype: int64

In [293]:
bom_df = filtered_movies

In [294]:
bom_df['gross_difference'] = bom_df['foreign_gross'] - bom_df['domestic_gross']
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year,gross_difference
0,Toy Story 3,BV,415000000.0,652000000.0,2010,237000000.0
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,357100000.0
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,368300000.0
3,Inception,WB,292600000.0,535700000.0,2010,243100000.0
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,275200000.0


In [295]:
average_difference = bom_df['gross_difference'].mean()
bom_df['foreign_gross'] = bom_df['foreign_gross'].fillna(bom_df['domestic_gross'] + average_difference)
bom_df = bom_df.drop(columns=['gross_difference'])



In [296]:
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1170 entries, 0 to 3226
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1170 non-null   object 
 1   studio          1170 non-null   object 
 2   domestic_gross  1170 non-null   float64
 3   foreign_gross   1170 non-null   float64
 4   year            1170 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 54.8+ KB


B) movie_info_df

In [297]:
movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [298]:
#drop columns with many missing values.
movie_info_df.drop(columns=['currency','box_office','studio'],inplace=True)


In [299]:
#fill null values with str'N/A'
movie_info_df.fillna('N/A',inplace=True)

In [300]:
movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1560 non-null   object
 2   rating        1560 non-null   object
 3   genre         1560 non-null   object
 4   director      1560 non-null   object
 5   writer        1560 non-null   object
 6   theater_date  1560 non-null   object
 7   dvd_date      1560 non-null   object
 8   runtime       1560 non-null   object
dtypes: int64(1), object(8)
memory usage: 109.8+ KB


C) reviews_df

In [301]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [302]:
reviews_df['rating'].unique()

array(['3/5', nan, 'C', '2/5', 'B-', '2/4', 'B', '3/4', '4/5', '4/4',
       '6/10', '1/4', '8', '2.5/4', '4/10', '2.0/5', '3/10', '7/10', 'A-',
       '5/5', 'F', '3.5/4', 'D+', '1.5/4', '3.5/5', '8/10', 'B+', '9/10',
       '2.5/5', '7.5/10', '5.5/10', 'C-', '1.5/5', '1/5', '5/10', 'C+',
       '0/5', '6', '0.5/4', 'D', '3.1/5', '3/6', '4.5/5', '0/4', '2/10',
       'D-', '7', '1/10', '3', 'A+', 'A', '4.0/4', '9.5/10', '2.5',
       '2.1/2', '6.5/10', '3.7/5', '8.4/10', '9', '1', '7.2/10', '2.2/5',
       '0.5/10', '5', '0', '2', '4.5', '7.7', '5.0/5', '8.5/10', '3.0/5',
       '0.5/5', '1.5/10', '3.0/4', '2.3/10', '4.5/10', '4/6', '3.5',
       '8.6/10', '6/8', '2.0/4', '2.7', '4.2/10', '5.8', '4', '7.1/10',
       '5/4', 'N', '3.5/10', '5.8/10', 'R', '4.0/5', '0/10', '5.0/10',
       '5.9/10', '2.4/5', '1.9/5', '4.9', '7.4/10', '1.5', '2.3/4',
       '8.8/10', '4.0/10', '2.2', '3.8/10', '6.8/10', '7.3', '7.0/10',
       '3.2', '4.2', '8.4', '5.5/5', '6.3/10', '7.6/10', '8.1/10',
  

ratings seem to have different values I need to make all the ratings out of 10 for valid analysis to be performed

In [303]:

def convert_to_10(rating):
     # Ensure input is a string
    grade_mapping = {'A+': 10, 'A': 9.5, 'A-': 9, 'B+': 8.5, 'B': 8, 'B-': 7.5, 'C+': 7, 'C': 6.5, 'C-': 6, 'D+': 5.5, 'D': 5,'D-': 4.5, 'F+':2.5,'F': 2,'F-': 1.5}
    
    if rating in grade_mapping:
        return grade_mapping[rating]
    else:
        return rating
    


             

In [304]:
reviews_df['rating out_of_ten'] =  reviews_df['rating'].apply(convert_to_10)
reviews_df['rating out_of_ten'].unique()


array(['3/5', nan, 6.5, '2/5', 7.5, '2/4', 8, '3/4', '4/5', '4/4', '6/10',
       '1/4', '8', '2.5/4', '4/10', '2.0/5', '3/10', '7/10', 9, '5/5', 2,
       '3.5/4', 5.5, '1.5/4', '3.5/5', '8/10', 8.5, '9/10', '2.5/5',
       '7.5/10', '5.5/10', 6, '1.5/5', '1/5', '5/10', 7, '0/5', '6',
       '0.5/4', 5, '3.1/5', '3/6', '4.5/5', '0/4', '2/10', 4.5, '7',
       '1/10', '3', 10, 9.5, '4.0/4', '9.5/10', '2.5', '2.1/2', '6.5/10',
       '3.7/5', '8.4/10', '9', '1', '7.2/10', '2.2/5', '0.5/10', '5', '0',
       '2', '4.5', '7.7', '5.0/5', '8.5/10', '3.0/5', '0.5/5', '1.5/10',
       '3.0/4', '2.3/10', '4.5/10', '4/6', '3.5', '8.6/10', '6/8',
       '2.0/4', '2.7', '4.2/10', '5.8', '4', '7.1/10', '5/4', 'N',
       '3.5/10', '5.8/10', 'R', '4.0/5', '0/10', '5.0/10', '5.9/10',
       '2.4/5', '1.9/5', '4.9', '7.4/10', '1.5', '2.3/4', '8.8/10',
       '4.0/10', '2.2', '3.8/10', '6.8/10', '7.3', '7.0/10', '3.2', '4.2',
       '8.4', '5.5/5', '6.3/10', '7.6/10', '8.1/10', '3.6/5', '2/6',
       

In [305]:
def normalize_rating(rating):
    if rating is None or rating == 'N' or rating == 'R' or rating == 'T':
        return None  # Return None for special cases
    
    if '/' in str(rating):
        parts = rating.split('/')
        if ' ' in parts[0]:
            return None  # Treat irregular ratings like '3 1/2' as null
        numerator = float(parts[0])
        denominator = float(parts[1])
        return (numerator / denominator) * 10  # Normalize to be out of ten
    
    if '-' in str(rating):
        return None  # Treat irregular ratings like '1-5' as null
    
    return float(rating)  # For other cases, simply convert to float

# Test the function
ratings = ['3/6', '3.8/5', '1-5', '7', 'N', '9.8', 'R', 'T', None, '6/10', 0.7, '3 1/2','4.0/4','2.1/2']
for rating in ratings:
    normalized = normalize_rating(rating)
    print(f"{rating}: {normalized}")


3/6: 5.0
3.8/5: 7.6
1-5: None
7: 7.0
N: None
9.8: 9.8
R: None
T: None
None: None
6/10: 6.0
0.7: 0.7
3 1/2: None
4.0/4: 10.0
2.1/2: 10.5


In [306]:
reviews_df['rating out_of_ten'] = reviews_df['rating out_of_ten'].apply(normalize_rating)

In [309]:
reviews_df['rating out_of_ten'].unique()

array([ 6.        ,  6.5       ,  4.        ,  7.5       ,  5.        ,
        8.        , 10.        ,  2.5       ,  6.25      ,  3.        ,
        7.        ,  9.        ,  2.        ,  8.75      ,  5.5       ,
        3.75      ,  8.5       ,  0.        ,  1.25      ,  6.2       ,
        4.5       ,  1.        ,  9.5       , 10.5       ,  7.4       ,
        8.4       ,  7.2       ,  4.4       ,  0.5       ,  7.7       ,
        1.5       ,  2.3       ,  6.66666667,  3.5       ,  8.6       ,
        2.7       ,  4.2       ,  5.8       ,  7.1       , 12.5       ,
        5.9       ,  4.8       ,  3.8       ,  4.9       ,  5.75      ,
        8.8       ,  2.2       ,  6.8       ,  7.3       ,  3.2       ,
       11.        ,  6.3       ,  7.6       ,  8.1       ,  3.33333333,
        1.8       ,  8.9       ,  8.2       ,  8.3       ,  4.33333333,
        4.1       ,  7.9       ,  8.7       ,  4.3       ,  9.6       ,
        1.7       ,  6.7       ,  9.2       ,  5.2       ,  3.7 

In [307]:
reviews_df.dropna(subset = ['rating out_of_ten'],inplace=True)

In [308]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40905 entries, 0 to 54431
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 40905 non-null  int64  
 1   review             35369 non-null  object 
 2   rating             40905 non-null  object 
 3   fresh              40905 non-null  object 
 4   critic             38925 non-null  object 
 5   top_critic         40905 non-null  int64  
 6   publisher          40685 non-null  object 
 7   date               40905 non-null  object 
 8   rating out_of_ten  40905 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 3.1+ MB


In [311]:
reviews_df.fillna('N/A',inplace=True)

In [312]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40905 entries, 0 to 54431
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 40905 non-null  int64  
 1   review             40905 non-null  object 
 2   rating             40905 non-null  object 
 3   fresh              40905 non-null  object 
 4   critic             40905 non-null  object 
 5   top_critic         40905 non-null  int64  
 6   publisher          40905 non-null  object 
 7   date               40905 non-null  object 
 8   rating out_of_ten  40905 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 3.1+ MB


D) imdb_df

In [313]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 4.5+ MB


In [318]:
imdb_df['genres'].fillna('N/A',inplace=True)

In [320]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73856 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 4.5+ MB
